In [17]:
import gradio as gr
import tensorflow as tf
import pickle
import keras_nlp

In [18]:
import tensorflow as tf
print(tf.__version__)

2.14.1


In [19]:
import tensorflow as tf
print(tf.version.VERSION)

2.14.1


In [21]:
MAX_SEQUENCE_LENGTH = 40


In [22]:
# Load the model
loaded_model = tf.keras.models.load_model("transformer_model")

# Load jw_vocab
with open("jw_vocab.pkl", "rb") as f:
    jw_vocab = pickle.load(f)

# Load rm_vocab
with open("rm_vocab.pkl", "rb") as f:
    rm_vocab = pickle.load(f)

# Load tokenizers
jw_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=jw_vocab, lowercase=False
)
rm_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=rm_vocab, lowercase=False
)

In [23]:
def translate_text(input_text):
    # Tokenize the input sentence
    input_tokens = jw_tokenizer([input_text]).to_tensor(
        shape=(None, MAX_SEQUENCE_LENGTH)
    )

    # Generate translation
    translated_tokens = keras_nlp.samplers.GreedySampler()(
        lambda prompt, cache, index: (
            loaded_model([input_tokens, prompt])[:, index - 1, :],
            None,
            cache,
        ),
        prompt=tf.concat(
            [
                tf.fill(
                    (tf.shape(input_tokens)[0], 1), rm_tokenizer.token_to_id("[START]")
                ),
                tf.fill(
                    (tf.shape(input_tokens)[0], MAX_SEQUENCE_LENGTH - 1),
                    rm_tokenizer.token_to_id("[PAD]"),
                ),
            ],
            axis=-1,
        ),
        end_token_id=rm_tokenizer.token_to_id("[END]"),
        index=1,
    )

    # Detokenize the translated tokens
    translated_text = rm_tokenizer.detokenize(translated_tokens)

    return (
        translated_text.numpy()[0]
        .decode("utf-8")
        .replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )


In [26]:
# Create Gradio Interface
iface = gr.Interface(
    fn=translate_text,
    inputs="text",
    outputs="text",
    live=True,
    title="Machine Transliteration",
    description="Translate Jawi  to Rumi script",
)

# Launch Gradio Interface
iface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [28]:
import streamlit as st
import tensorflow as tf
import pickle
import keras_nlp

MAX_SEQUENCE_LENGTH = 40

In [29]:
# Load the model
loaded_model = tf.keras.models.load_model("transformer_model")

# Load jw_vocab
with open("jw_vocab.pkl", "rb") as f:
    jw_vocab = pickle.load(f)

# Load rm_vocab
with open("rm_vocab.pkl", "rb") as f:
    rm_vocab = pickle.load(f)

# Load tokenizers
jw_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=jw_vocab, lowercase=False
)
rm_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=rm_vocab, lowercase=False
)

In [30]:
def translate_text(input_text):
    # Tokenize the input sentence
    input_tokens = jw_tokenizer([input_text]).to_tensor(
        shape=(None, MAX_SEQUENCE_LENGTH)
    )

    # Generate translation
    translated_tokens = keras_nlp.samplers.GreedySampler()(
        lambda prompt, cache, index: (
            loaded_model([input_tokens, prompt])[:, index - 1, :],
            None,
            cache,
        ),
        prompt=tf.concat(
            [
                tf.fill(
                    (tf.shape(input_tokens)[0], 1), rm_tokenizer.token_to_id("[START]")
                ),
                tf.fill(
                    (tf.shape(input_tokens)[0], MAX_SEQUENCE_LENGTH - 1),
                    rm_tokenizer.token_to_id("[PAD]"),
                ),
            ],
            axis=-1,
        ),
        end_token_id=rm_tokenizer.token_to_id("[END]"),
        index=1,
    )

    # Detokenize the translated tokens
    translated_text = rm_tokenizer.detokenize(translated_tokens)

    return (
        translated_text.numpy()[0]
        .decode("utf-8")
        .replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )

In [31]:
# Add your logo
logo_url = "https://static.wixstatic.com/media/a82f81_ee5d3c423b80433eaac59bb71b2bba16~mv2.png"  # Replace with the URL of your logo
st.image(logo_url, width=100)  # Adjust width as needed
# Streamlit app
st.title("Deep Learning for Classical Malay Text Transliteration")

# Input text box
input_text = st.text_input("Enter Jawi Text:")

# Translate button
if st.button("Translate"):
    if input_text:
        translation = translate_text(input_text)
        st.success("Translation:")
        st.write(translation)
    else:
        st.warning("Please enter some text to translate.")